### Challenge
* [ Dataset source](#source)
* [Dataset Features](#features)
* [Connect to any exchange rates API](#connect) (Sample - [Exchange Rates API](https://exchangeratesapi.io/)) to get exchange rates of Australia to New Zealand for the past 30 days into json output format
* [Pre process the data to manage any expected issues](#process)
* [Perform some data analysis](#perform)
    * Find the best and worst exchange rates for that time period
    * Calculate the average exchange rate for the month
* [Additional features or optimizations that you think are relevant and show off your skills.](#addition)


### Connect to exchange rates <a name="connect"></a>

In [ ]:
#Importing the packages
# pip install requests
import requests
import json
from datetime import datetime, timedelta
import pandas as pd

### Connect to exchange rates <a name="connect"></a>

Trying to retrieve the data from Exchange Rate API

In [ ]:
# Function to get the exchange rate for a specific date
def get_exchange_rate(api_key, date, base_currency='AUD', target_currency='NZD'):
    url = f"http://api.exchangeratesapi.io/v1/{date}"
    params = {
        'access_key': api_key,
        'base': base_currency,
        'symbols': target_currency
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error fetching data for {date}: {response.status_code}")
        print(f"Response: {response.json()}")
        return None

# Main function to execute the code
def main():
    api_key = "018023d5b0172abaa8faddef06410a09"
    end_date = datetime.today()
    start_date = end_date - timedelta(days=30)
    
    exchange_rates = {}
    
    for single_date in (start_date + timedelta(n) for n in range(30)):
        formatted_date = single_date.strftime('%Y-%m-%d')
        data = get_exchange_rate(api_key, formatted_date)
        
        if data:
            exchange_rates[formatted_date] = data['rates'][target_currency]
    
    with open('exchange_rates.json', 'w') as json_file:
        json.dump(exchange_rates, json_file, indent=4)
    
    print("Exchange rates data has been saved to exchange_rates.json")

# Run the main function
if __name__ == "__main__":
    main()
